<a href="https://colab.research.google.com/github/Vaibhavs10/scratchpad/blob/main/transformers_asr_pipeline_optimisations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A walkthrough of the pleathora of optimisations and nifty features offered via `pipeline` for Automatic Speech Recognition! 🤗

from yours truly - [Vaibhav (VB) Srivastav](https://twitter.com/reach_vb) 🤙

In [ ]:
!pip install -q --upgrade torch torchvision torchaudio
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q accelerate optimum bitsandbytes
!pip install -q ipython-autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 96.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 91.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%load_ext autotime

time: 338 µs (started: 2023-10-09 15:46:02 +00:00)


In [ ]:
import torch
from transformers import pipeline

time: 21.3 s (started: 2023-10-09 15:28:12 +00:00)


In [ ]:
pipe = pipeline("automatic-speech-recognition",
                "openai/whisper-large-v2",
                device="cuda:0")

time: 1min 26s (started: 2023-09-13 10:15:34 +00:00)


## Download a test file!

This is a 2 hour long audio snippet from a revdotcoms Earnings22 dataset.

In [ ]:
!wget https://huggingface.co/datasets/reach-vb/random-audios/resolve/main/4469669.mp3

--2023-09-13 14:50:17--  https://github.com/revdotcom/speech-datasets/raw/main/earnings22/media/4469669.mp3
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/revdotcom/speech-datasets/main/earnings22/media/4469669.mp3 [following]
--2023-09-13 14:50:17--  https://media.githubusercontent.com/media/revdotcom/speech-datasets/main/earnings22/media/4469669.mp3
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29628186 (28M) [audio/mpeg]
Saving to: ‘4469669.mp3’

4469669.mp3         100%[===================>]  28.25M  --.-KB/s    in 0.08s   

2023-09-13 14:50:19 (351 MB/s) - ‘4469669.mp

## Inference

By default Whisper is only capable of running inference for 30 second audios max.

# Chunking

In [ ]:
pipe("4469669.mp3", chunk_length_s=20)["text"][:200]

" Now it's time. May I start the presentation on Transforming Toshiba to Enhance Shareholder's Value and FY21 Second Quarter Consolidated Business Results. We are organizing this presentation session o"

time: 25min 2s (started: 2023-09-13 10:19:49 +00:00)


It took about 25 minutes and 2 seconds to run inference on a 2 hour long snippet.

Can we make this faster?

## Cue.. Batching

In [ ]:
pipe("4469669.mp3",
     chunk_length_s=20,
     batch_size=8)["text"][:200]

" Now it's time. May I start the presentation on Transforming Toshiba to Enhance Shareholder's Value and FY21 Second Quarter Consolidated Business Results. We are organizing this presentation session o"

time: 13min 36s (started: 2023-09-13 10:45:21 +00:00)


Voila!! It only took the `pipeline` 13 minutes and 36 seconds to run inference this time. We were able to reduce the inference time by 50%. 🚀

But, the real question is - can we make it even faster? 💨

## Half - Precision

In [ ]:
pipe = pipeline("automatic-speech-recognition",
                "openai/whisper-large-v2",
                torch_dtype=torch.float16,
                device="cuda:0")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


time: 2min 53s (started: 2023-10-09 15:06:34 +00:00)


In [ ]:
pipe("4469669.mp3",
     chunk_length_s=30,
     batch_size=8,
     return_timestamps=True)["text"][:200]

" Now it is time. May I start the presentation on Transforming Toshiba to Enhance Shareholder's Value and FY21 Second Quarter Consolidated Business Results. We are organizing this presentation session "

time: 5min 17s (started: 2023-10-09 15:20:36 +00:00)


## BetterTransformer

In [ ]:
pipe = pipeline("automatic-speech-recognition",
                "openai/whisper-large-v2",
                torch_dtype=torch.float16,
                device="cuda:0")

pipe.model = pipe.model.to_bettertransformer()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


time: 1min 48s (started: 2023-10-09 15:28:42 +00:00)


In [ ]:
pipe("4469669.mp3",
     chunk_length_s=30,
     batch_size=16,
     return_timestamps=True,)["text"][:200]

" Now it is time. May I start the presentation on Transforming Toshiba to Enhance Shareholder's Value and FY21 Second Quarter Consolidated Business Results. We are organizing this presentation session "

time: 3min 48s (started: 2023-10-09 15:31:00 +00:00)


In [ ]:
# import torch
# from transformers import pipeline
# from transformers import BitsAndBytesConfig

# quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16)

# pipe = pipeline("automatic-speech-recognition",
#                 "openai/whisper-large-v2",
#                 torch_dtype=torch.float16,
#                 device="cuda:0",
#                 quantization_config = quantization_config)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


TypeError: ignored

time: 2min 11s (started: 2023-10-09 15:47:10 +00:00)
